Anno 2022/2023

Progetto finale Introduzione alla Data Science

Matricola S5018707

In questo progetto lavorerò su due dataset effettuandone l'integrazione, la rappresentazione, e la predizione di certe statistiche attraverso lo studio

Tipi di dataset:

amazon_titles_1.csv, amazon_titles_2.csv
amazon_credits.csv

disney_titles_1.csv, disney_titles_2.csv
disney_credits.csv

Inizialmente importiamo pandas

In [1]:
import pandas as pd


Parto con l'esecuzione della read_csv per creare le tabelle per ogni dataframe

In [2]:
#Amazon

amazon_titles_1 = pd.read_csv("new_data/data/amazon/amazon_titles_1.csv")
amazon_titles_2 = pd.read_csv("new_data/data/amazon/amazon_titles_2.csv")

#Disney

disney_titles_1 = pd.read_csv("new_data/data/disney/disney_titles_1.csv")
disney_titles_2 = pd.read_csv("new_data/data/disney/disney_titles_2.csv")

# Voglio creare un file csv con tutti i titoli di Amazon e Disney



###############################################################################################################

Rappresentazione dei dati

Uso questo spazio per visualizzare le informazioni di ogni dataframe in modo da aiutarmi nello studio degli stessi

In [3]:
# Conteggi

# Valori nulli

#print(amazon_titles_1.isnull().sum())
#print(amazon_titles_2.isnull().sum())
#print(disney_titles_1.isnull().sum())
#print(disney_titles_2.isnull().sum())

#print(amazon_titles_combinata.isnull().sum())
#print(disney_titles_combinata.isnull().sum())

# Valori non nulli

#print(amazon_titles_1.notnull().sum())
#print(amazon_titles_2.notnull().sum())
#print(disney_titles_1.notnull().sum())
#print(disney_titles_2.notnull().sum())

#print(amazon_titles_combinata.notnull().sum())
#print(disney_titles_combinata.notnull().sum())

# Valori duplicati per colonna

#print(amazon_titles_1.duplicated().sum())
#print(amazon_titles_2.duplicated().sum())
#print(disney_titles_1.duplicated().sum())
#print(disney_titles_2.duplicated().sum())

#print(amazon_titles_combinata.duplicated().sum())
#print(disney_titles_combinata.duplicated().sum()) 

# Occorrenze di valori per colonna

#print(amazon_titles_1['type'].value_counts())
#print(amazon_titles_2['type'].value_counts())
#print(disney_titles_1['type'].value_counts())
#print(disney_titles_2['type'].value_counts())

#print(amazon_titles_combinata['date_added'].value_counts())
#print(disney_titles_combinata['type'].value_counts())


In [4]:
# Dettagli sui dataframe

# Prime 5 righe

#print(amazon_titles_1.head())
print(amazon_titles_2.head())
#print(disney_titles_1.head())
#print(disney_titles_2.head())

#print(amazon_titles_combinata.head())
#print(disney_titles_combinata.head())

# Informazioni generali

#print(amazon_titles_1.info())
#print(amazon_titles_2.info())
#print(disney_titles_1.info())
#print(disney_titles_2.info())

#print(amazon_titles_combinata.info())
#print(disney_titles_combinata.info())

# Descrizione

#print(amazon_titles_1.describe())
#print(amazon_titles_2.describe())
#print(disney_titles_1.describe())
#print(disney_titles_2.describe())

#print(amazon_titles_combinata.describe())
#print(disney_titles_combinata.describe())

# Dimensioni

#print(amazon_titles_1.shape)
#print(amazon_titles_2.shape)
#print(disney_titles_1.shape)
#print(disney_titles_2.shape)

#print(amazon_titles_combinata.shape)
#print(disney_titles_combinata.shape)

# Lista colonne

#print(amazon_titles_1.columns)
#print(amazon_titles_2.columns)
#print(disney_titles_1.columns)
#print(disney_titles_2.columns)

#print(amazon_titles_combinata.columns)
#print(disney_titles_combinata.columns)

  show_id   type                 title        director  \
0      s1  Movie   The Grand Seduction    Don McKellar   
1      s2  Movie  Take Care Good Night    Girish Joshi   
2      s3  Movie  Secrets of Deception     Josh Webber   
3      s4  Movie    Pink: Staying True  Sonia Anderson   
4      s5  Movie         Monster Maker    Giles Foster   

                                                cast         country  \
0     Brendan Gleeson, Taylor Kitsch, Gordon Pinsent          Canada   
1   Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar           India   
2  Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...   United States   
3  Interviews with: Pink, Adele, Beyoncé, Britney...   United States   
4  Harry Dean Stanton, Kieran O'Brien, George Cos...  United Kingdom   

       date_added  release_year rating duration                listed_in  \
0  March 30, 2021          2014    NaN  113 min            Comedy, Drama   
1  March 30, 2021          2018    13+  110 min     Drama, Interna

###############################################################################################################

Integrazione dei dataframe per ogni piattaforma

In [5]:
# Combino i due dataset usando la concat di pandas

# Per Amazon

amazon_titles_combinata = pd.concat([amazon_titles_1, amazon_titles_2])

# Per Disney

disney_titles_combinata = pd.concat([disney_titles_1, disney_titles_2])



Usando lo spazio apposito per la rappresentazione delle tabelle si nota che ci sono alcune colonne che rappresentano la stessa cosa con nomi diversi:

(country e production_countries, listed_in e genres, id e show_id, runtime e duration)

Quindi procedo con l'unione delle colonne che rappresentano la stessa cosa e la rimozione di quelle inutili

In [6]:
# Modifico le colonne con nomi diversi ma con lo stesso significato usando la funzione fillna di pandas

# (country e production_countries, listed_in e genres, id e show_id, runtime e duration)

# Amazon

amazon_titles_combinata['country'].fillna(amazon_titles_combinata['production_countries'], inplace=True)

amazon_titles_combinata['genres'].fillna(amazon_titles_combinata['listed_in'], inplace=True)

amazon_titles_combinata['id'].fillna(amazon_titles_combinata['show_id'], inplace=True)

amazon_titles_combinata['duration'].fillna(amazon_titles_combinata['runtime'], inplace=True)

# Disney

disney_titles_combinata['country'].fillna(disney_titles_combinata['production_countries'], inplace=True)

disney_titles_combinata['genres'].fillna(disney_titles_combinata['listed_in'], inplace=True)

disney_titles_combinata['id'].fillna(disney_titles_combinata['show_id'], inplace=True)

disney_titles_combinata['duration'].fillna(disney_titles_combinata['runtime'], inplace=True)


# Poi elimino le colonne che non mi servono più

amazon_titles_combinata.drop(['production_countries', 'listed_in', 'show_id', 'runtime'], axis=1, inplace=True)
disney_titles_combinata.drop(['production_countries', 'listed_in', 'show_id', 'runtime'], axis=1, inplace=True)



Dopo l'unione eseguo uno step di pulizia dei dati per eliminare le righe inutili usando lo spazio dedicato alla rappresentazione delle tabelle

Da tale analisi spuntano, per mio giudizio:

Comunemente ai due dataset

Rimuovo:
['seasons','date_added'] perchè ha troppi valori nulli per renderla utile
['description','imdb_id','cast','director'] dato che non rappresentano materia utile per uno studio

Modifico:
['type'] perchè guardando le occorrenze di valori della colonna si nota che ci sono quattro tipi('TV Show', 'Movie', 'MOVIE', 'SHOW'), ma per quello che indica la colonna è meglio usarne due('MOVIE', 'SHOW')

In [7]:
#Effettuo il drop delle colonne inutili


amazon_titles_combinata.drop(['description', 'imdb_id', 'seasons', 'date_added','cast','director'], axis=1, inplace=True)

disney_titles_combinata.drop(['description', 'imdb_id', 'seasons', 'date_added','cast','director'], axis=1, inplace=True)


# Modifica colonna type per definire i due valori assumibili da essa (MOVIE, SHOW)

amazon_titles_combinata['type'] = amazon_titles_combinata['type'].replace('Movie', 'MOVIE')
amazon_titles_combinata['type'] = amazon_titles_combinata['type'].replace('TV Show', 'SHOW')

disney_titles_combinata['type'] = disney_titles_combinata['type'].replace('Movie', 'MOVIE')
disney_titles_combinata['type'] = disney_titles_combinata['type'].replace('TV Show', 'SHOW')


amazon_titles_combinata.drop_duplicates(subset='id', inplace=True, keep='first')

disney_titles_combinata.drop_duplicates(subset='id', inplace=True, keep='first')


----------------------------------------------------------------------------------------------------------------
TRASFORMAZIONE
----------------------------------------------------------------------------------------------------------------

Purtroppo non capisco se siano realmente utili gli step di trasformazione assegnati dal progetto, penso che non avrebbe senso sostituire la colonna genres, dal momento che è una colonna che indica il genere del film, con una che ne indica solo il numero, sarebbe meglio aggiungerla a parte

Riguardo al creare colonne contenenti l'anno ed il mese di aggiunta, non ne trovo il senso visto che la colonna contiene una quantità irrisoria di valori, ma una parte del progetto lo richiede quindi procedo

In [13]:
# Aggiungo colonna che prende i valori della colonna genres, li mette in una lista e poi li conta inserendo il risultato in una nuova colonna chiamata genres_count

amazon_titles_combinata['genres_count'] = amazon_titles_combinata['genres'].tolist().count.apply(lambda x: x.split)


print(amazon_titles_combinata['genres_count'].value_counts())
print(amazon_titles_combinata['genres_count'])

Drama                                                           986
['drama']                                                       980
['comedy']                                                      807
['documentation']                                               597
Comedy                                                          536
                                                               ... 
['horror', 'scifi', 'action', 'thriller']                         1
['comedy', 'horror', 'european']                                  1
['drama', 'european', 'crime', 'history', 'romance']              1
['action', 'horror', 'scifi', 'thriller', 'war', 'european']      1
Anime, Comedy, Drama                                              1
Name: genres_count, Length: 2831, dtype: int64
0       ['drama', 'family', 'fantasy', 'comedy', 'roma...
1                        ['comedy', 'romance', 'western']
2                                       ['comedy', 'war']
3                          ['dr